# Week6 - Classifier Evaluation Lab

* Copy&paste your model for homework5 model
* Add grid search and train
* Compare performance
* Which one is better? Explain?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/classification/loan_status_data/loan_status.csv')

In [3]:
numerical_vars = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']
categorical_vars = ['Loan_ID','Gender','Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score
import seaborn as sns
from sklearn.linear_model import LogisticRegression

X=df.drop('Loan_Status', axis=1)
y=df['Loan_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=120)

print('Training set shape', X_train.shape, y_train.shape)
print('Test set shape', X_test.shape, y_test.shape)

Training set shape (304, 12) (304,)
Test set shape (77, 12) (77,)


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer

numerical_transformer=Pipeline([('imputer', SimpleImputer(strategy='median')),
                               ('scaler', StandardScaler())
                               ])

categorical_transformer=Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                                  ('onehot', OneHotEncoder(handle_unknown='ignore'))
                                 ])

In [6]:
X_train_numerical=X_train[numerical_vars]
numerical_transformed=numerical_transformer.fit_transform(X_train_numerical)
print('numerical pipeline transformed shape', numerical_transformed.shape)

X_train_categorical=X_train[categorical_vars]
categorical_transformed=categorical_transformer.fit_transform(X_train_categorical)
print('categorical pipeline transfromed shape', categorical_transformed.shape)

numerical pipeline transformed shape (304, 5)
categorical pipeline transfromed shape (304, 322)


In [7]:
processing_pipeline=ColumnTransformer(transformers=[('numerical', numerical_transformer, numerical_vars),
                                                    ('categorical', categorical_transformer, categorical_vars)])
print(processing_pipeline)

modeling_pipeline = Pipeline([
        ('data_processing', processing_pipeline),
        ('logreg', LogisticRegression(penalty=None))]
    )
modeling_pipeline

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['ApplicantIncome', 'CoapplicantIncome',
                                  'LoanAmount', 'Loan_Amount_Term',
                                  'Credit_History']),
                                ('categorical',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Loan_ID', 'Gender', 'Married', 'Dependents',
                

Pipeline(steps=[('data_processing',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['ApplicantIncome',
                                                   'CoapplicantIncome',
                                                   'LoanAmount',
                                                   'Loan_Amount_Term',
                                                   'Credit_History']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Loan_ID', 'Gender',
                                                   'Married', 'Dependents',
                                                   'Education', 'Self_Employed',
                                                   'Property_Area'])])),
                ('logreg', LogisticRegression(penalty=None))])

In [8]:
from sklearn.metrics import accuracy_score

modeling_pipeline.fit(X_train,y_train)

y_train_pred=modeling_pipeline.predict(X_train)
y_test_pred=modeling_pipeline.predict(X_test)

train_accuracy=accuracy_score(y_train,y_train_pred)
test_accuracy=accuracy_score(y_test,y_test_pred)

print('Accuracy on training set', train_accuracy)
print('Accuracy on test set', test_accuracy)

Accuracy on training set 1.0
Accuracy on test set 0.8701298701298701


In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

#defining parameter grid
param_grid = {
    'logreg__penalty': ['l1', 'l2'],
    'logreg__C': [0.001, 0.01, 0.1, 1, 10, 100]
}

#initializing GridSearchCV
grid_search = GridSearchCV(modeling_pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

#predicting on training and test set
y_train_pred_grid = best_model.predict(X_train)
y_test_pred_grid = best_model.predict(X_test)

#accuracy on training and test set
train_accuracy_grid = accuracy_score(y_train, y_train_pred_grid)
test_accuracy_grid = accuracy_score(y_test, y_test_pred_grid)

print('Accuracy on training set (with grid search):', train_accuracy_grid)
print('Accuracy on test set (with grid search):', test_accuracy_grid)

/Users/palak/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/palak/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-re

Accuracy on training set (with grid search): 0.8388157894736842
Accuracy on test set (with grid search): 0.8701298701298701


/Users/palak/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/palak/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
30 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the 

Both the models have same accuracy for test cases, the first model has accuracy 1.0 which is overfitting in whereas the second model which went through grid, search and train has accuracy 0.83, this represents a less overfitted model. 

so, the grid, search and train model is better in performace